In [10]:
import pandas as pd
import numpy as np
import re

In [4]:
from data_extraction import DataExtractor
from data_cleaning import DataCleaning

In [5]:
extractor = DataExtractor()
cleaning = DataCleaning()

In [6]:
extractor.extract_from_s3(bucket_name='data-handling-public', s3_key='products.csv', local_path='/opt/homebrew/Caskroom/miniconda/base/envs/mrdc/local_products.csv')

,Unnamed: 0,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
0,0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.6kg,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h
1,1,Tiffany's World Day Out At The Park,£12.99,0.48kg,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l
2,2,Tiffany's World Pups Picnic Playset,£7.00,590g,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v
3,3,Tiffany's World Wildlife Park Adventures,£12.99,540g,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,4,Cosatto Cosy Dolls Pram,£30.00,1.91kg,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a
...,...,...,...,...,...,...,...,...,...,...
1848,1848,Goodmans Qi Autosense In Car Phone Holder,£15.00,125g,diy,8185827751600,2020-08-28,3ff21cb1-8e90-499d-b5a0-6839fb83e70f,Still_avaliable,x4-2227164k
1849,1849,Goodmans Qi Wireless Charging Phone Stand,£15.00,134g,diy,9769724533935,2012-05-20,a7910f74-c2b6-4144-98b7-30ca68f34d2d,Still_avaliable,i3-8338545Z
1850,1850,RAC 12V Wet & Dry Vacuum,£18.00,820g,diy,3113855860327,1993-11-28,66d3efa0-f480-4da9-893b-e8aaff53e27b,Still_avaliable,n4-1642658b
1851,1851,Goodmans Qi Autosense In Car Phone Holder,£15.00,125g,diy,9195087467400,2016-09-20,efb472f1-8f00-448e-a874-9fa9812eeab5,Still_avaliable,A3-7619070S


In [7]:
print(raw_product_df)

NameError: name 'raw_product_df' is not defined

In [29]:
raw_product_df = extractor.extract_from_s3(bucket_name='data-handling-public', s3_key='products.csv', local_path='/opt/homebrew/Caskroom/miniconda/base/envs/mrdc/local_products.csv')

raw_product_df.to_csv('raw_product_df')

In [30]:
print(raw_product_df)

      Unnamed: 0                                 product_name product_price  \
0              0  FurReal Dazzlin' Dimples My Playful Dolphin        £39.99   
1              1          Tiffany's World Day Out At The Park        £12.99   
2              2          Tiffany's World Pups Picnic Playset         £7.00   
3              3     Tiffany's World Wildlife Park Adventures        £12.99   
4              4                      Cosatto Cosy Dolls Pram        £30.00   
...          ...                                          ...           ...   
1848        1848    Goodmans Qi Autosense In Car Phone Holder        £15.00   
1849        1849    Goodmans Qi Wireless Charging Phone Stand        £15.00   
1850        1850                     RAC 12V Wet & Dry Vacuum        £18.00   
1851        1851    Goodmans Qi Autosense In Car Phone Holder        £15.00   
1852        1852               Beldray Telescopic Ladder 2.9m        £69.00   

      weight        category            EAN  date_a

In [9]:
column_type = raw_product_df['weight'].dtype

print(column_type)

object


In [28]:
import pandas as pd
import re
import numpy as np

class DataCleaning:

    @staticmethod
    def convert_product_weights(product_df):
        converted_weights = []
        
        # Iterate over each weight value in the product_df 'weight' column
        for item in product_df['weight']:
            if isinstance(item, str):
                # Match patterns like '12 x 100' and compute the product
                matches = re.findall(r'(\d+)\s*x\s*(\d+)', item)
                if matches:
                    # Extract the two numbers, compute the product
                    num1, num2 = map(int, matches[0])
                    converted_weights.append(num1 * num2)  # Replace '12 x 100' with 1200
                elif item.endswith('kg'):
                    converted_weights.append(float(item[:-2]))  # Convert 'kg' to grams
                elif item.endswith('k'):
                    converted_weights.append(float(item[:-1]))  # Convert 'k' to grams
                elif item.endswith('g'):
                    converted_weights.append(float(item[:-1]) / 1000)  # Convert 'g' to float
                elif item.endswith('ml'):
                    converted_weights.append(float(item[:-2]) / 1000)  # Convert 'ml' to float
                else:
                    converted_weights.append(np.nan)  # Append NaN for unknown formats
            else:
                converted_weights.append(np.nan)  # Append NaN if item is not a string

        kg_weights = [f"{weight}kg" for weight in converted_weights]

        # Check if the length of converted_weights matches the length of product_df
        if len(converted_weights) != len(product_df):
            raise ValueError(f"Length of converted_weights ({len(converted_weights)}) does not match length of product_df ({len(product_df)})")

        # Assign the cleaned weights back to the DataFrame
        product_df['weight'] = kg_weights
        
        return product_df

# Example usage of the function
cleaning = DataCleaning()

# Call the function with S3 extraction (assuming the extractor function works)
product_df = extractor.extract_from_s3(
    bucket_name='data-handling-public', 
    s3_key='products.csv', 
    local_path='/opt/homebrew/Caskroom/miniconda/base/envs/mrdc/local_products.csv'
)

# Convert the product weights
cleaned_product_df = cleaning.convert_product_weights(product_df)

print(cleaned_product_df.head())

   Unnamed: 0                                 product_name product_price  \
0           0  FurReal Dazzlin' Dimples My Playful Dolphin        £39.99   
1           1          Tiffany's World Day Out At The Park        £12.99   
2           2          Tiffany's World Pups Picnic Playset         £7.00   
3           3     Tiffany's World Wildlife Park Adventures        £12.99   
4           4                      Cosatto Cosy Dolls Pram        £30.00   

   weight        category            EAN  date_added  \
0   1.6kg  toys-and-games  7425710935115  2005-12-02   
1  0.48kg  toys-and-games   487128731892  2006-01-09   
2  0.59kg  toys-and-games  1945816904649  1997-03-29   
3  0.54kg  toys-and-games  1569790890899  2013-03-20   
4  1.91kg  toys-and-games  7142740213920  2007-12-23   

                                   uuid          removed product_code  
0  83dc0a69-f96f-4c34-bcb7-928acae19a94  Still_avaliable  R7-3126933h  
1  712254d7-aea7-4310-aff8-8bcdd0aec7ff  Still_avaliable  C2-7

In [42]:

from data_cleaning import DataCleaning

class DataCleaning():
    def clean_products_data(self, products_df):
        # Remove rows where all values are Null
        products_df = products_df.dropna(how='all')
        
        # Remove duplicates
        products_df = products_df.drop_duplicates()

        # Create a new list for the 'removed' column
        new_removed = []

        for item in products_df['removed']:
            # Correct the typo "Still_avaliable" to "Still_available"
            if item == "Still_avaliable":  
                new_removed.append("Still_available")
            else:
                new_removed.append(item)

        # Assign the corrected 'removed' list back to the original DataFrame
        products_df['removed'] = new_removed

        new_product_names = []

        for item in products_df['product_name']:
            if isinstance(item, str):
                new_item = item.strip('"')
                new_product_names.append(new_item)
            else:
                new_product_names.append(item)

        products_df['product_name'] = new_product_names

        # Return the cleaned DataFrame
        return products_df


cleaning = DataCleaning()
cleaned_product_df2 = cleaning.clean_products_data(raw_product_df)
print(cleaned_product_df2.head())

   Unnamed: 0                                 product_name product_price  \
0           0  FurReal Dazzlin' Dimples My Playful Dolphin        £39.99   
1           1          Tiffany's World Day Out At The Park        £12.99   
2           2          Tiffany's World Pups Picnic Playset         £7.00   
3           3     Tiffany's World Wildlife Park Adventures        £12.99   
4           4                      Cosatto Cosy Dolls Pram        £30.00   

   weight        category            EAN  date_added  \
0   1.6kg  toys-and-games  7425710935115  2005-12-02   
1  0.48kg  toys-and-games   487128731892  2006-01-09   
2    590g  toys-and-games  1945816904649  1997-03-29   
3    540g  toys-and-games  1569790890899  2013-03-20   
4  1.91kg  toys-and-games  7142740213920  2007-12-23   

                                   uuid          removed product_code  
0  83dc0a69-f96f-4c34-bcb7-928acae19a94  Still_available  R7-3126933h  
1  712254d7-aea7-4310-aff8-8bcdd0aec7ff  Still_available  C2-7